In [ ]:
#pip install textblob

In [ ]:
#pip install pandas vaderSentiment

In [ ]:
#pip install praw

In [ ]:
#pip install tweepy nltk pandas

In [ ]:
#pip install yfinance

In [1]:
#pip install --upgrade yfinance


## Reddit

In [16]:
import praw
import json
from datetime import datetime


# Initialize PRAW with your credentials
reddit = praw.Reddit(client_id='IFRFuqUUyPaFsqKCKl9TaA',
                     client_secret='Adp9lyh4Y79swxDgxhLfnbUE29Nzcg',
                     user_agent='script:stockX:v1.0 (by /u/Perman_27)')

# Choose the subreddit you're interested in
subreddits_to_analyze = ['wallstreetbets', 'stocks', 'investing']

# Define a list to store the scraped data
all_posts_data = []

# Define the time frame for historical discussions
time_frame = 'month'  # can be 'day', 'week', 'month', 'year', 'all'

# Loop through each subreddit
for subreddit_name in subreddits_to_analyze:
    subreddit = reddit.subreddit(subreddit_name)
    
    # Use the search function to look for posts containing 'AAPL' or 'Apple'
    search_query = 'AAPL OR Apple'
    for post in subreddit.search(search_query, time_filter=time_frame, limit=100):  
        post_dict = {
            "title": post.title,
            "subreddit": subreddit_name,
            "content": post.selftext,
            "score": post.score,  # Proxy for user engagement
            "upvote_ratio": post.upvote_ratio,
            "num_comments": post.num_comments,
            "created_utc": post.created_utc,
            "date": datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),  # Human-readable date
            "flair": post.link_flair_text,
            "tickers": [word for word in post.title.split() if word.startswith("$")]
        }
        
        # Scrape top comments for each post (limit to top 10 for brevity)
        post.comments.replace_more(limit=0)
        comments_data = []
        for comment in post.comments.list()[:10]:
            comments_data.append({
                "content": comment.body,
                "score": comment.score,
                "created_utc": comment.created_utc,
                "date": datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')  # Human-readable date
            })
        
        # Add comments to the post dictionary
        post_dict["comments"] = comments_data
        
        # Append post data to the list
        all_posts_data.append(post_dict)

# Optionally save the scraped data to a JSON file
with open('reddit/apple_posts_data.json', 'w') as json_file:
    json.dump(all_posts_data, json_file, indent=4)

## Yahoo Finance

In [25]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import logging
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import csv
import os

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def extract_recommendation_trends(soup):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Locate the section containing analyst ratings
    ratings_section = soup.find('section', {'data-test': 'analyst-ratings'})
    
    # Initialize a dictionary to store the recommendation categories
    recommendation_trends = {}
    
    if ratings_section:
        # Locate the list items representing each recommendation category
        recommendation_list = ratings_section.find_all('li')
        
        for recommendation in recommendation_list:
            # The text for the recommendation is the second child (after the <i> element)
            category_text = recommendation.text.strip()
            
            # Assuming the structure is "<icon> Category Name", split and get the last element
            category = category_text.split()[-1]
            recommendation_trends[category] = category_text

    return recommendation_trends

def extract_stock_details(soup):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract company name
    company_name = soup.find('h1').text.strip()

    # Extract current price details
    current_price = soup.find('fin-streamer', {'data-field': 'regularMarketPrice'}).text
    price_change = soup.find('fin-streamer', {'data-field': 'regularMarketChange'}).text
    percentage_change = soup.find('fin-streamer', {'data-field': 'regularMarketChangePercent'}).text.strip("()")

    # Extract after-hours price details
    after_hours_price = soup.find('fin-streamer', {'data-field': 'postMarketPrice'}).text
    after_hours_change = soup.find('fin-streamer', {'data-field': 'postMarketChange'}).text
    after_hours_change_percent = soup.find('fin-streamer', {'data-field': 'postMarketChangePercent'}).text.strip("()")

    # Extract dividend information
    dividend_info = soup.find('p', text='Dividend').find_next_sibling('p').text

    # Construct a dictionary with the extracted information
    stock_details = {
        'Company Name': company_name,
        'Current Price': current_price,
        'Price Change': price_change,
        'Percentage Change': percentage_change,
        'After Hours Price': after_hours_price,
        'After Hours Change': after_hours_change,
        'After Hours Change Percent': after_hours_change_percent,
        'Dividend Information': dividend_info,
    }
    
    return stock_details

# Fetches dividend data using yfinance
def fetch_dividend_data(ticker):
    stock = yf.Ticker(ticker)
    dividends = stock.dividends
    return dividends

# Fetches historical earnings data using yfinance
def fetch_earnings_data(ticker):
    stock = yf.Ticker(ticker)
    earnings = stock.earnings
    return earnings

# Fetches options data using yfinance
def fetch_options_data(ticker):
    stock = yf.Ticker(ticker)
    options_dates = stock.options
    options_data = {}
    for date in options_dates:
        options_data[date] = stock.option_chain(date)
    return options_data

# Fetches historical stock prices using yfinance
def fetch_stock_prices(ticker, period='1mo'):
    stock = yf.Ticker(ticker)
    hist = stock.history(period=period)
    return hist[['Open', 'Close', 'High', 'Low', 'Volume']]

# Fetches financial reports using yfinance
def fetch_financial_reports(ticker):
    stock = yf.Ticker(ticker)
    balance_sheet = stock.balance_sheet
    income_statement = stock.financials
    cashflow = stock.cashflow
    return balance_sheet, income_statement, cashflow
# Saves data to CSV
def save_data_to_csv(data, filename, folder='data'):
    if not os.path.isdir(folder):
        os.mkdir(folder)
    data.to_csv(os.path.join(folder, f'{filename}.csv'))

def scrape_yahoo_finance(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}/analysis"  # URL to the analysis page
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    # ... Retry strategy setup remains the same ...
    
    session = requests.Session()
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    try:
        response = session.get(url, headers=headers)
        response.raise_for_status()
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Pass the soup object to the extraction functions
        stock_details = extract_stock_details(soup)
        recommendation_trends = extract_recommendation_trends(soup)
        
        return stock_details, recommendation_trends
    except requests.RequestException as e:
        logger.error(f'An error occurred while scraping Yahoo Finance: {e}')
        return None, None

# Call the revised scrape function
stock_details, recommendation_trends = scrape_yahoo_finance(ticker_symbol)
print(stock_details)
print(recommendation_trends)


# Usage example:
ticker_symbol = 'AAPL'
stock_details, recommendation_trends = scrape_yahoo_finance(ticker_symbol)
print(stock_details)
print(recommendation_trends)

# Fetch analyst ratings by scraping
def fetch_all_earnings_tables(url, table_class, timeout=5):
    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Set up retry strategy
    retry_strategy = Retry(
        total=3,
        status_forcelist=[429, 500, 502, 503, 504],
        method_whitelist=["HEAD", "GET", "OPTIONS"],
        backoff_factor=1
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session = requests.Session()
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    all_tables_data = []

    try:
        response = session.get(url, headers=headers, timeout=timeout)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            earnings_tables = soup.find_all('table', {'class': table_class})
            
            for table in earnings_tables:
                earnings_rows = table.find_all('tr')
                if earnings_rows:
                    headers = [header.text.strip() for header in earnings_rows[0].find_all('th')]
                    table_data = []
                    for row in earnings_rows[1:]:
                        cols = row.find_all('td')
                        if cols:
                            table_data.append({headers[i]: cols[i].text.strip() for i in range(len(cols))})
                    all_tables_data.append(table_data)
                else:
                    logger.info('No rows found in one of the earnings tables.')
            return all_tables_data
        else:
            logger.error(f'Failed to retrieve the webpage, status code: {response.status_code}')
            for table in table_data:
                 print(table)
            return []

    except requests.RequestException as e:
        logger.error(f'An error occurred: {e}')
        return []

    
def save_data_to_csv(data, filename, folder='data'):
    if not os.path.isdir(folder):
        os.mkdir(folder)
    data.to_csv(os.path.join(folder, f'{filename}.csv'))
    
    
    
def save_to_csv(data, filename):
    if not os.path.isdir('data'):
        os.mkdir('data')
    data.to_csv(f'data/{filename}.csv')
    
# Usage:
ticker_symbol = 'AAPL'  # Replace with your ticker

# Fetch and save dividend data
dividends = fetch_dividend_data(ticker_symbol)
save_data_to_csv(dividends, f'{ticker_symbol}_dividends')

# Fetch and save analyst recommendations
recommendations = fetch_analyst_recommendations(ticker_symbol)
save_data_to_csv(recommendations, f'{ticker_symbol}_analyst_recommendations')

# Fetch and save historical earnings data
earnings = fetch_earnings_data(ticker_symbol)
save_data_to_csv(earnings, f'{ticker_symbol}_earnings')

# Fetch and save options data
options_data = fetch_options_data(ticker_symbol)
for date, option_data in options_data.items():
    save_data_to_csv(option_data.calls, f'{ticker_symbol}_options_calls_{date}')
    save_data_to_csv(option_data.puts, f'{ticker_symbol}_options_puts_{date}')


# Usage:
prices = fetch_stock_prices(ticker_symbol, period='1mo')
save_to_csv(prices, f'{ticker_symbol}_prices')


# Usage:
balance_sheet, income_statement, cashflow = fetch_financial_reports(ticker_symbol)
save_to_csv(balance_sheet, f'{ticker_symbol}_balance_sheet')
save_to_csv(income_statement, f'{ticker_symbol}_income_statement')
save_to_csv(cashflow, f'{ticker_symbol}_cashflow')


def save_list_of_dicts_to_csv(data, filename):
    if not data:
        return

    keys = data[0].keys()
    with open(f'data/{filename}.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)

# Usage:



def save_scraped_tables(all_tables_data, base_filename):
    for idx, table_data in enumerate(all_tables_data):
        save_list_of_dicts_to_csv(table_data, f'{base_filename}_table_{idx+1}')

# Usage:
all_tables_data = fetch_all_earnings_tables(url, 'W(100%) M(0) BdB Bdc($seperatorColor) Mb(25px)')
save_scraped_tables(all_tables_data, ticker_symbol)



NameError: name 'adapter' is not defined

In [38]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import logging
import csv
import os
from time import sleep
from requests.packages.urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter


# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)



def extract_recommendation_trends(soup):
    # Initialize a dictionary to store the recommendation categories
    recommendation_trends = {}
    
    # Locate the section containing analyst ratings
    ratings_section = soup.find('section', {'data-test': 'analyst-ratings'})
    
    if ratings_section:
        # Locate the list items representing each recommendation category
        recommendation_list = ratings_section.find_all('li')
        
        for recommendation in recommendation_list:
            # The text for the recommendation is the entire text of the list item
            category_text = recommendation.text.strip()
            
            # The category name is the text of the recommendation minus the icons and numbers
            category = " ".join(category_text.split()[1:])  # Skip the icon symbol (first element)
            recommendation_trends[category] = category_text

    return recommendation_trends

def extract_stock_details(soup):
    # Extract company name
    company_name = soup.find('h1').text.strip()

    # Extract current price details
    current_price = soup.find('fin-streamer', {'data-field': 'regularMarketPrice'}).text
    price_change = soup.find('fin-streamer', {'data-field': 'regularMarketChange'}).text
    percentage_change = soup.find('fin-streamer', {'data-field': 'regularMarketChangePercent'}).text.strip("()")

    # Extract after-hours price details
    after_hours_price = soup.find('fin-streamer', {'data-field': 'postMarketPrice'}).text
    after_hours_change = soup.find('fin-streamer', {'data-field': 'postMarketChange'}).text
    after_hours_change_percent = soup.find('fin-streamer', {'data-field': 'postMarketChangePercent'}).text.strip("()")

    # Extract dividend information
    dividend_info = ""
    dividend_section = soup.find('p', text=lambda x: x and 'Dividend' in x)
    if dividend_section and dividend_section.find_next_sibling('p'):
        dividend_info = dividend_section.find_next_sibling('p').text

    # Construct a dictionary with the extracted information
    stock_details = {
        'Company Name': company_name,
        'Current Price': current_price,
        'Price Change': price_change,
        'Percentage Change': percentage_change,
        'After Hours Price': after_hours_price,
        'After Hours Change': after_hours_change,
        'After Hours Change Percent': after_hours_change_percent,
        'Dividend Information': dividend_info,
    }
    
    return stock_details

# Scrape Yahoo Finance for stock details and recommendation trends
def scrape_yahoo_finance(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}/analysis"
    headers = {'User-Agent': 'Mozilla/5.0'}
    session = requests.Session()
    response = session.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        stock_details = extract_stock_details(soup)
        recommendation_trends = extract_recommendation_trends(soup)
        return stock_details, recommendation_trends
    else:
        logger.error(f"Failed to scrape Yahoo Finance for {ticker}. Status code: {response.status_code}")
        return {}, {}
    
def fetch_all_earnings_tables(url, table_class, timeout=5):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    
    # Set up a session with retries
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))

    all_tables_data = []

    try:
        response = session.get(url, headers=headers, timeout=timeout)
        response.raise_for_status()

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            earnings_tables = soup.find_all('table', {'class': table_class})
            
            for table in earnings_tables:
                earnings_rows = table.find_all('tr')
                if earnings_rows:
                    headers = [header.text.strip() for header in earnings_rows[0].find_all('th')]
                    table_data = []
                    for row in earnings_rows[1:]:
                        cols = row.find_all('td')
                        if cols:
                            row_data = {headers[i]: cols[i].text.strip() for i in range(len(cols))}
                            table_data.append(row_data)
                    all_tables_data.append(table_data)
            return all_tables_data
        else:
            logger.error(f'Failed to retrieve the webpage, status code: {response.status_code}')
            return []

    except requests.RequestException as e:
        logger.error(f'An error occurred: {e}')
        return []
    finally:
        session.close()

# Other imports and function definitions remain unchanged...

# Define a function to save stock details to CSV
def save_stock_details_to_csv(stock_details, filename, folder='data'):
    if not os.path.isdir(folder):
        os.mkdir(folder)
    
    filepath = os.path.join(folder, f'{filename}.csv')
    with open(filepath, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=stock_details.keys())
        writer.writeheader()
        writer.writerow(stock_details)
    
    logger.info(f"Stock details saved to {filepath}")

# Define a function to save recommendation trends to CSV
def save_recommendation_trends_to_csv(recommendation_trends, filename, folder='data'):
    if not os.path.isdir(folder):
        os.mkdir(folder)
    
    filepath = os.path.join(folder, f'{filename}.csv')
    with open(filepath, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Category', 'Details'])
        for category, details in recommendation_trends.items():
            writer.writerow([category, details])
    
    logger.info(f"Recommendation trends saved to {filepath}")

def save_scraped_tables(all_tables_data, base_filename, folder='data'):
    if not os.path.isdir(folder):
        os.mkdir(folder)
    
    for idx, table_data in enumerate(all_tables_data):
        filename = f"{base_filename}_table_{idx+1}.csv"
        file_path = os.path.join(folder, filename)
        with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
            dict_writer = csv.DictWriter(csvfile, fieldnames=table_data[0].keys())
            dict_writer.writeheader()
            dict_writer.writerows(table_data)
        
        logger.info(f"Table data saved to {file_path}")

def main(ticker_symbol):
    # Scrape Yahoo Finance for stock details and recommendation trends
    stock_details, recommendation_trends = scrape_yahoo_finance(ticker_symbol)
    
    # Save the scraped stock details and recommendation trends
    save_stock_details_to_csv(stock_details, f"{ticker_symbol}_stock_details")
    save_recommendation_trends_to_csv(recommendation_trends, f"{ticker_symbol}_recommendation_trends")

    # Log the scraped stock details and recommendation trends
    logger.info(f"Stock Details for {ticker_symbol}: {stock_details}")
    logger.info(f"Recommendation Trends for {ticker_symbol}: {recommendation_trends}")
    
    # Fetch and save all earnings tables data
    earnings_tables_url = f"https://finance.yahoo.com/quote/{ticker_symbol}/analysis"
    earnings_tables_class = 'W(100%) M(0) BdB Bdc($seperatorColor) Mb(25px)'
    all_tables_data = fetch_all_earnings_tables(earnings_tables_url, earnings_tables_class)
    if all_tables_data:
        save_scraped_tables(all_tables_data, ticker_symbol)
    else:
        logger.info(f"No table data found for {ticker_symbol}.")

# Execute the main function
if __name__ == "__main__":
    ticker = 'AAPL'  # Replace with your ticker symbol
    main(ticker)


/var/folders/nr/0m3_2w416k95_79fx2rpjb7h0000gn/T/ipykernel_33632/2689235472.py:55: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  dividend_section = soup.find('p', text=lambda x: x and 'Dividend' in x)
INFO:__main__:Stock details saved to data/AAPL_stock_details.csv
INFO:__main__:Recommendation trends saved to data/AAPL_recommendation_trends.csv
INFO:__main__:Stock Details for AAPL: {'Company Name': 'Apple Inc. (AAPL)', 'Current Price': '4,358.34', 'Price Change': '+40.56', 'Percentage Change': '+0.94%', 'After Hours Price': '176.65', 'After Hours Change': '+0.01', 'After Hours Change Percent': '+0.00%', 'Dividend Information': ''}
INFO:__main__:Recommendation Trends for AAPL: {}
INFO:__main__:Table data saved to data/AAPL_table_1.csv
INFO:__main__:Table data saved to data/AAPL_table_2.csv
INFO:__main__:Table data saved to data/AAPL_table_3.csv
INFO:__main__:Table data saved to data/AAPL_table_4.csv
INFO:__main__:Table data saved t

In [22]:
def fetch_analyst_recommendations(ticker):
    url = f'https://finance.yahoo.com/quote/{ticker}/analysis'
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # You would need to inspect the webpage to find the correct selector for the recommendations
        recommendations_table = soup.find('table', {'class': 'recommendations'})
        if recommendations_table:
            rows = recommendations_table.find_all('tr')
            data = []
            for row in rows[1:]:  # Skip header row
                cols = row.find_all('td')
                if cols:
                    data.append({
                        'Date': cols[0].text.strip(),
                        'Rating': cols[1].text.strip(),
                        # Add more columns as needed based on the structure of the table
                    })
            return data
        else:
            print("Couldn't find the recommendations table.")
            return []
    else:
        print("Couldn't retrieve the page.")
        return []

# Usage
ticker_symbol = 'AAPL'
recommendations = fetch_analyst_recommendations(ticker_symbol)
print(recommendations)

Couldn't find the recommendations table.
[]


In [11]:
import pandas as pd
import json
from textblob import TextBlob

# Sentiment Analyzer function using TextBlob
def sentiment_analyzer(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

# Load Reddit data
with open('reddit/apple_posts_data.json', 'r') as file:
    reddit_data = pd.json_normalize(json.load(file))

# Load stock prices data
prices_data = pd.read_csv('data/AAPL_prices.csv')

# Preprocess and merge data
def preprocess_and_merge(reddit_df, prices_df):
    # Convert 'created_utc' to dates
    reddit_df['Date'] = pd.to_datetime(reddit_df['created_utc'], unit='s').dt.date
    
    # Assign sentiment scores to Reddit posts
    reddit_df['sentiment'] = reddit_df['content'].apply(sentiment_analyzer)

    # Aggregate the number of posts and average sentiment per day
    discussion_volume = reddit_df.groupby('Date').agg({
        'sentiment': 'mean',
        'title': 'size'  # Assuming 'title' column exists for counting posts
    }).rename(columns={'title': 'post_count'}).reset_index()

    # Ensure 'Date' in prices_df is a datetime object
    prices_df['Date'] = pd.to_datetime(prices_df['Date']).dt.date

    # Merge the datasets on the 'Date' column
    merged_df = pd.merge(discussion_volume, prices_df, on='Date', how='inner')

    # Add any additional merging requirements here, such as tickers if needed
    # Sentiment Score Association and Volume of Discussion Correlation
    # Ensure data types match before merging
    #reddit_df['post_count'] = reddit_df['post_count'].astype(float)
    #prices_df['Volume'] = prices_df['Volume'].astype(float)
    #merged_df['discussion_volume'] = reddit_df['post_count']
    # merged_df = pd.merge(merged_df, additional_data, on=['Date', 'ticker'], how='inner')

    return merged_df

# Perform preprocessing and merging
merged_data = preprocess_and_merge(reddit_data, prices_data)

# Calculate correlations if merging is successful
if merged_data is not None:
    correlation_discussion_trading = merged_data['post_count'].corr(merged_data['Volume'])
    correlation_price_sentiment = merged_data['sentiment'].corr(merged_data['Close'])
    correlation_volume_sentiment = merged_data['sentiment'].corr(merged_data['Volume'])
    # ... rest of your code ...

    print(f"Correlation between discussion volume and trading volume: {correlation_discussion_trading}")
    print(f"Correlation between sentiment and closing price: {correlation_price_sentiment}")
    print(f"Correlation between sentiment and trading volume: {correlation_volume_sentiment}")
    
    # Save the new merged data to a CSV file
    merged_data.to_csv('merged_reddit_finance_data.csv', index=False)
    print("Merged data saved to 'merged_reddit_finance_data.csv'.")


else:
    print("Data merging was unsuccessful. Please check the data formats and try again.")


Correlation between discussion volume and trading volume: 0.41376442111383244
Correlation between sentiment and closing price: -0.2902920283344923
Correlation between sentiment and trading volume: 0.05095853574382976
Merged data saved to 'merged_reddit_finance_data.csv'.


In [18]:
import pandas as pd
import os
import json
from pathlib import Path

# Function to analyze a single CSV file
def analyze_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        analyze_dataframe(df, file_path)
    except Exception as e:
        print(f'Could not read {file_path}: {e}')

# Function to analyze a single JSON file with nested structure details
def analyze_json(file_path):
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
            df = pd.json_normalize(data, sep='_')
            analyze_dataframe(df, file_path)
            if isinstance(data, dict):
                print_nested_json_structure(data, level=0)
            elif isinstance(data, list) and data:
                print_nested_json_structure(data[0], level=0)  # Assumes first element is representative of the list
    except Exception as e:
        print(f'Could not read {file_path}: {e}')

# Helper function to print nested JSON structure
def print_nested_json_structure(data, level):
    indent = '  ' * level
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, dict) or isinstance(value, list):
                print(f'{indent}{key}:')
                print_nested_json_structure(value, level + 1)
            else:
                print(f'{indent}{key}: {type(value).__name__}')
    elif isinstance(data, list):
        print(f'{indent}List of {len(data)} items: {type(data[0]).__name__}')
        if data and isinstance(data[0], (dict, list)):
            print_nested_json_structure(data[0], level + 1)

# General function to analyze DataFrames
def analyze_dataframe(df, file_path):
    column_details = df.dtypes.apply(lambda x: x.name).to_dict()
    num_rows, num_columns = df.shape
    nan_values = df.isna().sum().to_dict()

    print(f'File: {file_path}')
    print(f'Columns and Types: {column_details}')
    print(f'Number of Rows: {num_rows}')
    print(f'Number of Columns: {num_columns}')
    print(f'NaN values per column: {nan_values}')
    print('-----------------------------------\n')

# Function to traverse directories and find CSV and JSON files
def find_files(directory):
    pathlist = Path(directory).rglob('*.*')
    for path in pathlist:
        # Analyze files based on extension
        if path.suffix.lower() == '.csv':
            analyze_csv(str(path))
        elif path.suffix.lower() == '.json':
            analyze_json(str(path))

# Replace 'your_directory_path' with the path to the directory containing your CSV and JSON files
your_directory_path = './'
find_files(your_directory_path)


File: merged_reddit_finance_data.csv
Columns and Types: {'Date': 'object', 'sentiment': 'float64', 'post_count': 'int64', 'Open': 'float64', 'Close': 'float64', 'High': 'float64', 'Low': 'float64', 'Volume': 'int64'}
Number of Rows: 3
Number of Columns: 8
NaN values per column: {'Date': 0, 'sentiment': 0, 'post_count': 0, 'Open': 0, 'Close': 0, 'High': 0, 'Low': 0, 'Volume': 0}
-----------------------------------

File: reddit/apple_posts_data.json
Columns and Types: {'title': 'object', 'subreddit': 'object', 'content': 'object', 'score': 'int64', 'upvote_ratio': 'float64', 'num_comments': 'int64', 'created_utc': 'float64', 'date': 'object', 'flair': 'object', 'tickers': 'object', 'comments': 'object'}
Number of Rows: 107
Number of Columns: 11
NaN values per column: {'title': 0, 'subreddit': 0, 'content': 0, 'score': 0, 'upvote_ratio': 0, 'num_comments': 0, 'created_utc': 0, 'date': 0, 'flair': 29, 'tickers': 0, 'comments': 0}
-----------------------------------

title: str
subreddit: 